In [1]:
import boto3
import json
from decimal import Decimal

In [3]:
class DecimalEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Decimal):
            return float(obj)
        return super(DecimalEncoder, self).default(obj)

def get_table_head(table_name, limit=5, region_name='us-east-1'):
    """
    Read the first few records from a DynamoDB table
    """
    print(f"\n--- First {limit} records from {table_name} table ---")
    
    # Create a session and resource
    dynamodb = boto3.resource('dynamodb', region_name=region_name)
    table = dynamodb.Table(table_name)
    
    try:
        # Scan with a limit
        response = table.scan(Limit=limit)
        
        # Check if there are any items
        if 'Items' in response and response['Items']:
            print(f"Found {len(response['Items'])} items:")
            
            for i, item in enumerate(response['Items']):
                print(f"\nItem {i+1}:")
                print(json.dumps(item, indent=2, cls=DecimalEncoder))
        else:
            print(f"No items found in the {table_name} table.")
            
        print(f"ScannedCount: {response.get('ScannedCount', 0)}")
        print(f"Count: {response.get('Count', 0)}")
        
    except Exception as e:
        print(f"Error scanning {table_name}: {e}")

# Script to read head from all three tables

In [14]:
# get_table_head("Nodes")
get_table_head("Distances")
# get_table_head("Readings")


--- First 5 records from Distances table ---
Found 4 items:

Item 1:
{
  "fromNodeId": "node0",
  "distanceId": "ded4156",
  "distance": 9.9,
  "toNodeId": "node1",
  "timestamp": "2025-04-14T17:23:15Z",
  "timestamp_toNodeId": "2025-04-14T17:23:15Z_node1"
}

Item 2:
{
  "fromNodeId": "node0",
  "distanceId": "dc8fd11",
  "distance": 10.08,
  "toNodeId": "node2",
  "timestamp": "2025-04-14T17:23:15Z",
  "timestamp_toNodeId": "2025-04-14T17:23:15Z_node2"
}

Item 3:
{
  "fromNodeId": "node0",
  "distanceId": "d8d6af3",
  "distance": 14.05,
  "toNodeId": "node3",
  "timestamp": "2025-04-14T17:23:15Z",
  "timestamp_toNodeId": "2025-04-14T17:23:15Z_node3"
}

Item 4:
{
  "fromNodeId": "node0",
  "distanceId": "d42f1a5",
  "distance": 7.2,
  "toNodeId": "node4",
  "timestamp": "2025-04-14T17:23:15Z",
  "timestamp_toNodeId": "2025-04-14T17:23:15Z_node4"
}
ScannedCount: 4
Count: 4


In [17]:
def count_table_entries(region_name='us-east-1'):
    """
    Count the number of entries in each DynamoDB table
    """
    dynamodb = boto3.resource('dynamodb', region_name=region_name)
    tables = ['Nodes', 'Readings', 'Distances']
    
    print("\n--- Table Entry Counts ---")
    for table_name in tables:
        try:
            table = dynamodb.Table(table_name)
            response = table.scan(Select='COUNT')
            count = response['Count']
            
            # Handle pagination if there are more than 1MB of results
            while 'LastEvaluatedKey' in response:
                response = table.scan(
                    Select='COUNT',
                    ExclusiveStartKey=response['LastEvaluatedKey']
                )
                count += response['Count']
                
            print(f"{table_name}: {count} entries")
        except Exception as e:
            print(f"Error counting {table_name}: {e}")
    
    return None

def clear_database(confirm=True, region_name='us-east-1'):
    """
    Clear all entries from all DynamoDB tables
    This is a destructive operation and requires confirmation
    """
    if confirm:
        confirmation = input("WARNING: This will delete ALL records from ALL tables. Type 'DELETE' to confirm: ")
        if confirmation != "DELETE":
            print("Operation cancelled.")
            return False
    
    dynamodb = boto3.resource('dynamodb', region_name=region_name)
    tables = ['Nodes', 'Readings', 'Distances']
    total_deleted = 0
    
    print("\n--- Clearing Database Tables ---")
    for table_name in tables:
        try:
            table = dynamodb.Table(table_name)
            # Get primary key information
            table_description = table.meta.client.describe_table(TableName=table_name)
            key_schema = table_description['Table']['KeySchema']
            
            # Scan for all items
            response = table.scan()
            items = response.get('Items', [])
            deleted_count = 0
            
            # Handle pagination
            while 'LastEvaluatedKey' in response:
                response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
                items.extend(response.get('Items', []))
            
            # Delete each item
            print(f"Deleting {len(items)} items from {table_name}...")
            for item in items:
                # Create a key dictionary based on the key schema
                key = {attr['AttributeName']: item[attr['AttributeName']] 
                       for attr in key_schema}
                table.delete_item(Key=key)
                deleted_count += 1
                
                # Print progress every 100 items
                if deleted_count % 100 == 0:
                    print(f"  Deleted {deleted_count} items...")
            
            print(f"Completed: {deleted_count} items deleted from {table_name}")
            total_deleted += deleted_count
            
        except Exception as e:
            print(f"Error clearing {table_name}: {e}")
    
    print(f"\nTotal items deleted: {total_deleted}")
    return True

# Example usage
# Count entries in all tables


In [30]:
count_table_entries()


--- Table Entry Counts ---
Nodes: 5 entries
Readings: 57 entries
Distances: 40 entries


In [ ]:
# RUN TO CLEAR DBH
clear_database()


--- Clearing Database Tables ---
Deleting 5 items from Nodes...
Completed: 5 items deleted from Nodes
Deleting 14 items from Readings...
Completed: 14 items deleted from Readings
Deleting 56 items from Distances...
Completed: 56 items deleted from Distances

Total items deleted: 75


True